# Preparing workspace

In [ ]:
#load "Paket.fsx"
 
Paket.Dependencies.Install """
framework: netstandard2.0
generate_load_scripts: true
storage: none
source https://nuget.org/api/v2
nuget CNTK.CPUOnly
nuget Newtonsoft.Json
nuget MathNet.Numerics
nuget MathNet.Numerics.FSharp
"""

In [4]:
#r "netstandard"
#r @"..\bin\Cntk.Core.Managed-2.6.dll"
#load "../Graphviz/.paket/load/main.group.fsx"

In [5]:
open System
open System.IO

Environment.GetEnvironmentVariable("PATH")
|> fun path -> sprintf "%s%c%s" path (Path.PathSeparator) (Path.GetFullPath("bin"))
|> fun path -> Environment.SetEnvironmentVariable("PATH", path)

open CNTK
DeviceDescriptor.UseDefaultDevice().Type
|> printfn "Congratulations, you are using CNTK for: %A" 

Congratulations, you are using CNTK for: CPU


In [6]:
let device = CNTK.DeviceDescriptor.CPUDevice
let dataType = CNTK.DataType.Float
let initialization = CNTKLib.GlorotUniformInitializer(1.0)
let input_dim, num_output_classes = 2,2

#load "../fsx/CntkHelpers.fsx"
open CntkHelpers

let input = Variable.InputVariable(shape [|input_dim|], dataType, "Features")
let label = Variable.InputVariable(shape [|num_output_classes|], dataType, "Labels")
let z = fullyConnectedClassifierNet input [5] num_output_classes CNTKLib.Sigmoid

In [7]:
open System.Collections.Generic

// combine filters to single lambda
let inline (<&>) (filter1 : 'T -> bool) (filter2 : 'T -> bool) =
        fun (t:'T) -> filter1 t && filter2 t

let decomposeFunction (root: Function) = 
        
        let visited = Dictionary<string, Function>()            

        let rec search (f: Function) = 
            visited.Add(f.Uid, f)
        
            seq {   yield! f.Inputs
                    yield! f.Outputs
                    yield  Var f.RootFunction }
            |> Seq.map (fun v -> v.Owner)
            |> Seq.filter (fun f -> f <> null && visited.ContainsKey(f.Uid) = false)            
            |> Seq.iter search

        search root
    
        visited

In [8]:
let decz = decomposeFunction z

In [9]:
decz
|> Seq.cast<KeyValuePair<string,Function>> 
|> Seq.map(fun pair -> pair.Value.AsString()) 
|> Array.ofSeq
|> Array.iter(printfn "%s")

Composite(Plus): Input('Features', [2], [*, #]) -> Output('+', [2], [*, #])
Plus: Output('@', [2], [*, #]) -> Output('+', [2], [*, #])
Times: Output('StableSigmoid10_Output_0', [5], [*, #]) -> Output('@', [2], [*, #])
StableSigmoid: Output('+', [5], [*, #]) -> Output('StableSigmoid10_Output_0', [5], [*, #])
Plus: Output('@', [5], [*, #]) -> Output('+', [5], [*, #])
Times: Input('Features', [2], [*, #]) -> Output('@', [5], [*, #])


In [10]:
type NodeInfo = {Property: string; Value: string}

let nodeInfo (tensor: obj) = 
    
    let matchTypeName (prop:System.Reflection.PropertyInfo) = 
        try
            match prop.PropertyType.FullName with        
            | "CNTK.Function" -> 
                (prop.GetValue(tensor) :?> CNTK.Function).Uid |> sprintf "fun %s"
            | "CNTK.Variable" -> 
                (prop.GetValue(tensor) :?> CNTK.Variable).Uid |> sprintf "var %s"
            | (list:string) when list.Contains("[CNTK.Variable") -> 
                prop.GetValue(tensor) :?> CNTK.Variable seq 
                |> Seq.map(fun (v: Variable) -> "var " + v.Uid + " " + v.AsString()) 
                |> Seq.reduce(sprintf "%s, %s")
            | (list:string) when list.Contains("[CNTK.Axis") -> 
                prop.GetValue(tensor) :?> CNTK.Axis seq 
                |> Seq.map(fun (a:Axis) -> a.AsString()) 
                |> Seq.reduce(sprintf "%s, %s")
            | "CNTK.NDShape" -> (prop.GetValue(tensor) :?> CNTK.NDShape).AsString()
            | _ -> prop.GetValue(tensor) |> string
        with exn ->
            sprintf "%s: %s -> %s" prop.Name prop.PropertyType.Name exn.Message
    
    seq {
        yield "Type", tensor.GetType().FullName
        match tensor with 
        | :? CNTK.Function -> 
            yield "AsString()", (tensor :?> Function).AsString()
            yield! typeof<Function>.GetProperties()
            |> Array.map (fun prop -> 
                prop.Name, matchTypeName prop)
        | :? CNTK.Variable -> 
            yield "AsString", (tensor :?> Variable).AsString()
            yield! typeof<Variable>.GetProperties()
            |> Array.map (fun prop -> 
                prop.Name, matchTypeName prop)
        | _ ->  (tensor.GetType().FullName) 
                |> sprintf "Unexpected property %s" 
                |> failwith
    } |> Seq.map (fun (k,v) -> { Property= k; Value= v } )


//nodeInfo decz.["Plus18"].Output |> Util.Table
//decz.["Plus18"].Output |> nodeInfo |> Util.Table

//decz.["Plus18"].AsString()


In [ ]:
open Newtonsoft.Json

decz.["Plus18"].Output |> nodeInfo |> JsonConvert.SerializeObject

In [ ]:
// https://github.com/Microsoft/CNTK/blob/master/bindings/python/cntk/logging/graph.py

let extractGraphVizDotNotation (f: Function) = 
        let varText (v:Variable) = (if String.IsNullOrEmpty v.Name then v.Uid else v.Name) + "\\n" + v.Shape.AsString()
        let funText (f: Function) = if String.IsNullOrEmpty f.Name then f.Uid else f.Name
    
        let varLabel (v: Variable) = sprintf "%s [label=\"%s\"];" v.Uid (varText v)
        let funLabel (f: Function) = sprintf "%s [label=\"%s\"];" f.Uid (funText f)

        let varShape (v: Variable) =
            match v with
            | _ when v.IsInput -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsOutput -> sprintf "%s [shape=invhouse, color=gray];" v.Uid
            | _ when v.IsPlaceholder -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsParameter -> sprintf "%s [shape=diamond, color=green];" v.Uid
            | _ when v.IsConstant -> sprintf "%s [shape=rectangle, color=lightblue];" v.Uid
            | _ -> sprintf "%s [shape=circle, color=purple];" v.Uid

        let funShape (f: Function) = 
            match f with 
            | _ when f.IsComposite -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, peripheries=2];" f.Uid
            | _ when f.IsPrimitive -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, size=0.6];" f.Uid
            | _ -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=4];" f.Uid

        let varEdges (f: Function) (v: Variable) = 
            let inputIndex = f.Inputs |> Seq.map (fun v -> v.Uid) |> Set
            let outputIndex = f.Outputs |> Seq.map (fun v -> v.Uid) |> Set
            match inputIndex.Contains(v.Uid), outputIndex.Contains(v.Uid) with 
            | true, _ when v.IsParameter -> sprintf "%s -> %s [label=\"input param\"];" v.Uid f.Uid |> Some
            | _, true when v.IsParameter -> sprintf "%s -> %s [label=\"output param\"];" f.Uid v.Uid|> Some
            | true, _ -> sprintf "%s -> %s [label=input];" v.Uid f.Uid|> Some 
            | _, true -> 
                if f.Outputs.Count = 1 && f.Output.Uid = v.Uid then
                    printfn "%s %s %s %s" f.Outputs.[0].Uid f.Output.Uid v.Uid v.Owner.Output.Uid
                    sprintf "%s -> %s [label=output];" f.Uid v.Owner.Output.Uid |> Some
                else sprintf "%s -> %s [label=output];" f.Uid v.Uid|> Some
            //| _ when v.IsParameter -> sprintf "%s -> %s [label=param];" f.Uid v.Uid|> Some
            | _ -> None //sprintf "%s -> %s;" f.Uid v.Uid

        let vars = Seq.append f.Inputs f.Outputs
        let funs = seq { 
                yield f
                yield f.RootFunction;
                yield! vars |> Seq.filter (fun v -> v.Owner |> isNull |> not) |> Seq.map (fun v -> v.Owner) 
            } 

        seq {        
            if f.Uid <> f.RootFunction.Uid then yield sprintf "%s -> %s [label=\"root function\"];" f.RootFunction.Uid f.Uid
            yield! vars |> Seq.map varShape
            yield! vars |> Seq.map varLabel
            yield! vars |> Seq.map (varEdges f) |> Seq.choose id
            yield! funs |> Seq.map funLabel 
            yield! funs |> Seq.map funShape
        } |> Seq.distinct


In [ ]:
       
let createGraphVizDiagram (f:Function) =
    f 
    |> decomposeFunction 
    |> Seq.cast<KeyValuePair<string,Function>> 
    |> Seq.map(fun pair -> pair.Value) 
    //|> Seq.distinct
    //|> Seq.where(fun v -> v.Owner)
//     |> Seq.where (function Fun _ -> true | _ -> false)
    |> Seq.where (fun f -> f.IsComposite |> not && (f.IsPrimitive && f.IsBlock |> not && f.Outputs.Count = 1) )
    |> Seq.collect extractGraphVizDotNotation
    |> Seq.distinct //|> Array.ofSeq |> Array.filter (fun str -> str.Contains("->")) |> Array.sort
    |> Seq.sort
    
    
//createGraphVizDiagram z |> Array.ofSeq

createGraphVizDiagram z 
|> Seq.reduce(sprintf "%s\n%s")
|> sprintf "digraph { %s }"
|> renderDot

In [3]:
type Tensor = 
| F of Function 
| V of Variable
with 
    member x.Uid = match x with F f -> f.Uid | V v -> v.Uid
    member x.AsString = match x with F f -> f.AsString() | V v -> v.AsString()

let decomposeTensor (root: Tensor) = 
        
        let visited = Dictionary<string, Tensor>()            

        let rec search (f: Tensor) = 
            visited.Add(f.Uid, f)
        
            seq {   
                match f with
                | F f' -> 
                    yield! f'.Inputs |> Seq.map V
                    yield! f'.Outputs |> Seq.map V
                    yield  f'.RootFunction |> F 
                | V v' -> 
                    yield f
                    if v'.Owner |> isNull |> not then 
                        yield v'.Owner |> F
            }
            |> Seq.filter (fun f -> visited.ContainsKey(f.Uid) |> not)
            |> Seq.iter search

        search root
    
        visited

The type 'Function' is not defined.
The type 'Variable' is not defined.
The type 'Function' is not defined.
The type 'Variable' is not defined.
The field, constructor or member 'Uid' is not defined.
The field, constructor or member 'Uid' is not defined.
The field, constructor or member 'AsString' is not defined. Maybe you want one of the following:
   ToString
The field, constructor or member 'AsString' is not defined. Maybe you want one of the following:
   ToString
The value or constructor 'Dictionary' is not defined. Maybe you want one of the following:
   dict
Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
The field, constructor or member 'Inputs' is not defined.
The field, constructor or member 'Outputs' is not defined.
The field, constructor or member 'RootFunction' is not defined.
The field, construct

In [2]:
decomposeTensor (F z) 
|> Seq.cast<KeyValuePair<string,Tensor>> 
|> Seq.map (fun t -> 
        t.Value 
        |> function 
        | F f -> f |> nodeInfo 
        | V v -> v |> nodeInfo
        |> JsonConvert.SerializeObject
        |> sprintf "\t\"%s\": %s" t.Value.Uid )
|> Seq.take 3
|> Seq.reduce(sprintf "%s,\n%s")        
|> printf "{ \n%s\n}"


The value or constructor 'decomposeTensor' is not defined.
The type 'KeyValuePair' is not defined.
The pattern discriminator 'F' is not defined.
The value, namespace, type or module 'JsonConvert' is not defined.
The field, constructor or member 'Uid' is not defined.

<div style="width: 100%;min-height: 600px; border: solid lightblue 1px; border-radius: 15px; position: relative">
    <div id="graph" style="width: 100%; height: 100%; background-color: red"></div>
    <div id="info" style="width: 300px; height: 500px; border-radius: 15px; box-shadow: 5px 10px #888888;position: absolute; top: 15px; right: 15px"></div>
</div>

In [ ]:
@"<script src='../../../d3-jupyter/dist/bundle.js'></script>" |> Util.Html |> Display

In [1]:
let initGraph infoPath graphPath = 
    sprintf "<script>$(document).trigger('INIT_D3', ['%s','%s']);</script>" infoPath graphPath
    |> Util.Html
    
let initInfo infoJson = 
    sprintf "<script>$(document).trigger('INIT_GRAPH_INFO', ['%s']);</script>" infoJson
    |> Util.Html
    
let renderDot dotNotation = 
    sprintf "<script>$(document).trigger('RENDER_GRAPH', [`%s`]);</script>" dotNotation
    |> Util.Html
    
let renderSeries (digraphs : string[]) = 
    digraphs
    |> Array.reduce(sprintf "%s','%s")
    |> sprintf "<script>$(document).trigger('RENDER_SERIES', [['%s']]);</script>"
    |> Util.Html   
    
"""<script>$(document).on("NODE_CLICKED", 
    (e,d,i,n)=>{
        console.log(e,d,i,n);
});</script>""" |> Util.Html
    
initGraph "#graph"    

<fun:it@24> : (string -> HtmlOutput)

In [ ]:
let createGraphWithInfo (z: Function) =
    
    let nodeInfo = 
    
    initGraph "#info" "#graph" |> ignore
    initInfo 

    createGraphVizDiagram z 
    |> Seq.reduce(sprintf "%s\n%s")
    |> sprintf "digraph { %s }"
    |> renderDot

In [ ]:
"""<script>
        var dot = `
  digraph {
    Combine1787 -> CompositeFunction1788 [label="root function"];
Combine1787 [label="Combine1787"];
Combine1787 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1789 -> CompositeFunction1790 [label="root function"];
Combine1789 [label="Combine1789"];
Combine1789 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1791 -> CompositeFunction1792 [label="root function"];
Combine1791 [label="Combine1791"];
Combine1791 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1793 -> CompositeFunction1794 [label="root function"];
Combine1793 [label="Combine1793"];
Combine1793 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1795 -> CompositeFunction1796 [label="root function"];
Combine1795 [label="Combine1795"];
Combine1795 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1797 -> CompositeFunction1798 [label="root function"];
Combine1797 [label="Combine1797"];
Combine1797 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1799 -> CompositeFunction1800 [label="root function"];
Combine1799 [label="Combine1799"];
Combine1799 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1801 -> CompositeFunction1802 [label="root function"];
Combine1801 [label="Combine1801"];
Combine1801 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1803 -> CompositeFunction1804 [label="root function"];
Combine1803 [label="Combine1803"];
Combine1803 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
  }`;
        var viz = d3.select("#graph")
            .graphviz()
            //.logEvents(true)
            .on("initEnd", function () {

                //viz.engine("circo");
                viz.renderDot(dot)
                    .on("end", function () {

                        let svg = d3.select("svg");//.attr("width", "1750pt").attr("height","1000pt");

                        let defs = svg.append("defs");

                        let filter = defs
                            .append("filter")
                            .attr("id", "shadow")
                            .attr("x", "-50%")
                            .attr("y", "-50%")
                            .attr("width", "200%")
                            .attr("height", "200%");
                        filter.append("feGaussianBlur")
                            .attr("in", "SourceAlpha")
                            .attr("stdDeviation", 3)
                            .attr("result", "blur");
                        filter
                            .append("feOffset")
                            .attr("in", "blur")
                            .attr("dx", 3)
                            .attr("dy", 3);
                        filter.append("feComponentTransfer")
                            .append("feFuncA").attr("type", "linear")
                            .attr("slope", 0.35);

                        var merge = filter.append("feMerge");
                        merge.append("feMergeNode");
                        merge.append("feMergeNode").attr("in", "SourceGraphic");

                        d3.selectAll(".node ellipse, .node polygon")
                            .style("fill", "white")
                            .on("mouseover", (d, i, n) => d3.select(n[i]).style("filter", "url(#shadow)"))
                            .on("mouseout", (d, i, n) => d3.select(n[i]).style("filter", null));
                    });
            });
    </script>""" |> Util.Html |> Display

In [ ]:
"""<script>
        var canvas = d3.select("#graph")
                        .append("svg")
                        .attr("width", 500)
                        .attr("height", 500);
        var circle = canvas.append("circle")
                        .attr("cx",250)
                        .attr("cy", 250)
                        .attr("r", 50)
                        .attr("fill", "red");
    </script>
""" |> Util.Html

In [ ]:
createGraphVizDiagram z |> Seq.reduce(sprintf "%s\t%s\n")

In [ ]:
input.ToFunction().Uid

In [ ]:
"""<script>$(document).on("NODE_CLICKED", 
    (e,d,i,n)=>{
        console.log(e,d,i,n);
});</script>""" |> Util.Html

In [ ]:
"../../../d3-jupyter/dist/bundle.js" |> System.IO.Path.GetFullPath